In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import re
import numpy as np

In [2]:
def crawl_kompas(url, path):
    df = pd.DataFrame(columns=['judul', 'tanggal', 'kategori', 'isi_berita', 'ringkasan', 'url'])
    try:
        for page in range(200, 351):
            print(f'==================== PAGE : {page} ====================')
            response = requests.get(f'{url}{page}?page=all', timeout=10)
            if response.status_code == 200:
                links = []
                response.raise_for_status()
        
                soup = BeautifulSoup(response.text, 'html.parser')
                # links = soup.find_all('a')
                divs = soup.find_all('div', class_='articleItem')
        
                # print(f'Ketemu URL : {url}{page}')
        
                for div in divs:
                    link = div.find('a')
                    if link and link.get('href'):
                        # print(link['href'])
                        links.append(link['href']+'?page=all')

                for link in links:
                    try:
                        pagex = requests.get(f'{link}?page=all', timeout=10)
                        soupx = BeautifulSoup(pagex.text, 'html.parser')
                        
                        # print(link)

                        # JUDUL
                        judul = ''
                        titles = soupx.find('h1', class_='read__title')
                        for t in titles:
                            judul = t
                        print(f'Judul: {judul}')

                        # TANGGAL
                        meta_date_name = 'content_PublishedDate'
                        meta_date = soupx.find('meta', attrs={'name': meta_date_name})
                        if meta_date and meta_date.get('content'):
                            date = meta_date['content']
                            # print(f'Tanggal: {date}')
                        else:
                            print(f'Meta denga name {meta_date_name} tidak ditemukan')

                        # KATEGORI
                        meta_category_name = 'content_category'
                        meta_category = soupx.find('meta', attrs={'name': meta_category_name})
                        if meta_category and meta_category.get('content'):
                            category = meta_category['content']
                            # print(f'Kategori: {category}')
                        else:
                            print(f'Meta dengan name {meta_category_name} tidak ditemukan')
                        
                        # ISI BERITA
                        # container = soupx.find('div', class_='col-bs9-7')
                        content_outer_container = soupx.find('div', class_='read__content')
                        content_inner_container = content_outer_container.find('div', class_='clearfix')
                        paragraphs = content_inner_container.find_all('p')

                        isi_berita_list = []
                        for p in paragraphs:
                            # print(f'Isi berita: {p.text}')
                            isi_berita_list.append(p.text)
                        isi_berita = ' '.join(isi_berita_list)
                        # print(f'Isi berita: {isi_berita}')
    
                        # print(isi_berita, '\n')

                        # RINGKASAN
                        meta_summary_name = 'twitter:description'
                        meta_summary = soupx.find('meta', attrs={'name': meta_summary_name})
        
                        if meta_summary and meta_summary.get('content'):
                            summary = meta_summary['content'].replace(' Halaman all?page=all', '')
                            # print(f'Ringkasan: {summary}')
                        else:
                            print(f'Meta tag dengan name {meta_summary_name} tidak ditemukan')

                        # URL
                        # print(f'URL: {link}')
                        
                        # print('\n')

                        df.loc[len(df)] = [judul, date, category, isi_berita, summary, link]
                    except requests.exceptions.RequestException as e:
                        print(f"Gagal mengakses link {link}: {e}")

                os.makedirs(os.path.join(path, 'Data'), exist_ok=True)
                df.to_csv(os.path.join(path, 'Data', 'kompas.csv'), index=False)
                print('Data sudah OK!')

            else:
                print(f"Gagal mengakses halaman. Kode status: {response.status_code}")

    except requests.exceptions.RequestException as e:
        print(f'Terjadi kesalahan saat mengakses {url}: {e}')

    return df

PATH = 'D:/SEMESTER 7 (2024)/PPW/TUGAS AKHIR PPW'
url = 'https://indeks.kompas.com/?page='
crawl_kompas(url, PATH)

==================== PAGE : 200 ====================
Judul: Ibu Dua Anak di Gresik Tewas Dianiaya Suami, Sempat Kabur hingga Teras Tetangga
Judul: KPU Sikka Mulai Distribusikan Logistik Pilkada untuk Wilayah Kepulauan
Judul: Pawai Iklim di Semarang, Suara Anak Muda untuk Lingkungan
Judul: Biomassa Jadi Faktor Penting Pengurangan Emisi dan Peningkatan Ekonomi Rakyat
Judul: Prabowo dan Presiden MBZ Saksikan Pertukaran MoU RI-UEA
Judul: Bupati Gunungkidul Kecewa, 2 ASN yang Selingkuh Kembali Diaktifkan
Judul: 3 Hari Baku Tembak di Pakistan Tewaskan 82 Orang, 156 Terluka
Judul: 5 Perusahaan Minyak Dituding Hasilkan Plastik 1.000 Kali Lebih Banyak, Benarkah?
Judul: Ririe Fairus Doakan Ayus Setia dengan Nissa Sabyan
Judul: BNPB: Korban Tewas Banjir Bandang di Deli Serdang 4 Orang
Judul: Kawasan Padat Penduduk di Jatinegara Kebakaran, Diduga karena Ledakan Tabung Gas
Judul: Cara Cek Ukuran Foto dan Video WhatsApp yang Sudah Terkirim
Judul: Status Siaga Darurat Bencana Hidrometeorologi DIY Dip

,judul,tanggal,kategori,isi_berita,ringkasan,url
0,"Ibu Dua Anak di Gresik Tewas Dianiaya Suami, S...",2024-11-24 15:59:36,Regional,KOMPAS.com - Pertengkaran suami istri di Gresi...,Pertengkaran suami istri berujung tragis di Gr...,https://surabaya.kompas.com/read/2024/11/24/15...
1,KPU Sikka Mulai Distribusikan Logistik Pilkada...,2024-11-24 15:59:17,Regional,"SIKKA, KOMPAS.com – Komisi Pemilihan Umum (KPU...",KPU Sikka mulai mendistribusikan logistik pilk...,https://regional.kompas.com/read/2024/11/24/15...
2,"Pawai Iklim di Semarang, Suara Anak Muda untuk...",2024-11-24 15:57:40,Regional,"SEMARANG, KOMPAS.com - Ratusan anak muda menge...",Ratusan anak muda mengenakan baju putih berkum...,https://regional.kompas.com/read/2024/11/24/15...
3,Biomassa Jadi Faktor Penting Pengurangan Emisi...,2024-11-24 15:54:41,Money,"JAKARTA, KOMPAS.com – Pemanfaatan biomassa seb...","Menurut PLN, salah satu contoh keberhasilan pe...",https://money.kompas.com/read/2024/11/24/15544...
4,Prabowo dan Presiden MBZ Saksikan Pertukaran M...,2024-11-24 15:52:12,News,"JAKARTA, KOMPAS.com - Indonesia dan Uni Emirat...",Indoneisa dan UEA sepakat menandatangani kerja...,https://nasional.kompas.com/read/2024/11/24/15...
...,...,...,...,...,...,...
2257,"PPN Mau Naik Jadi 12 Persen, Bagaimana Nasib P...",2024-11-21 13:07:08,Otomotif,"JAKARTA, KOMPAS.com – Penjualan mobil di pasar...",Kenaikan PPN bakal langsung berdampak pada men...,https://otomotif.kompas.com/read/2024/11/21/13...
2258,"YouTube Gaming Recap 2024 Dirilis, Kilas Balik...",2024-11-21 13:07:00,Tekno,KOMPAS.com - YouTube setiap tahun merilis fitu...,YouTube merilis Gaming Recap 2024. Fitur ini m...,https://tekno.kompas.com/read/2024/11/21/13070...
2259,Kisah Satu Keluarga di Jepara Bangun Jembatan ...,2024-11-21 13:06:41,Regional,"JEPARA, KOMPAS.com - Keluarga Sunardi (70), pe...",Joko pun menyebukan sudah berkoordinasi dengan...,https://regional.kompas.com/read/2024/11/21/13...
2260,Wapres Tinjau Makan Bergizi Gratis di SMKN 3 T...,2024-11-21 13:06:29,News,"JAKARTA, KOMPAS.com - Wakil Presiden (Wapres) ...",Gibran Rakabuming Raka meninjau pelaksanaan uj...,https://nasional.kompas.com/read/2024/11/21/13...
